<a href="https://colab.research.google.com/github/Kushagra3219/CNN-Models/blob/main/Model_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

# -------------------------
# Load MNIST
# -------------------------
(x_train_raw, y_train), (x_test_raw, y_test) = tf.keras.datasets.mnist.load_data()

# Define target image size
TARGET_IMG_SIZE = (96, 96)
BATCH_SIZE = 32

# Function to preprocess images for the dataset
def preprocess_image(image, label):
    image = tf.image.resize(tf.image.grayscale_to_rgb(tf.expand_dims(image, -1)), TARGET_IMG_SIZE)
    image = preprocess_input(tf.cast(image, tf.float32))
    return image, label

# Create tf.data.Dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_raw, y_train))
train_dataset = train_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Create tf.data.Dataset for testing
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_raw, y_test))
test_dataset = test_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("Train Dataset created.")
print("Test Dataset created.")

# -------------------------
# Pretrained ResNet50
# -------------------------
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(TARGET_IMG_SIZE[0], TARGET_IMG_SIZE[1], 3)
)

# Freeze pretrained layers
base_model.trainable = False

# -------------------------
# Add MNIST classifier head
# -------------------------
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -------------------------
# Train
# -------------------------
history = model.fit(
    train_dataset, # Use the dataset here
    validation_data=test_dataset, # Use the test dataset for validation if no separate validation set is needed
    epochs=3,
    verbose=2
)

# -------------------------
# Evaluate
# -------------------------
loss, acc = model.evaluate(test_dataset, verbose=0)
print(f"\nTest Accuracy: {acc:.4f}")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train Dataset created.
Test Dataset created.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 3, 3, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,851,274 (90.99 MB)

 Trainable params: 263,562 (1.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/3
1875/1875 - 80s - 43ms/step - accuracy: 0.9187 - loss: 0.2545 - val_accuracy: 0.9631 - val_loss: 0.1116
Epoch 2/3
1875/1875 - 69s - 37ms/step - accuracy: 0.9582 - loss: 0.1306 - val_accuracy: 0.9693 - val_loss: 0.0939
Epoch 3/3
1875/1875 - 50s - 26ms/step - accuracy: 0.9635 - loss: 0.1121 - val_accuracy: 0.9676 - val_loss: 0.0960

Test Accuracy: 0.9676
